# VisoMaster Runtime Setup Guide

Welcome! This notebook guides you through setting up and running the VisoMaster application within this Vast.ai environment.

**Instructions:**
1.  Run the cells in each section sequentially by clicking the 'Run' button (▶) or pressing `Shift+Enter`.
2.  Read the instructions and output in each cell carefully.
3.  Some steps require you to upload files (like `requirements.txt` or the VisoMaster code) to the `/workspace` directory using the Jupyter Lab file browser on the left.
4.  This notebook uses interactive widgets. Please wait for commands to finish before proceeding.

**Environment:**
*   **Persistent Storage:** Files saved in `/workspace` will persist across instance restarts.
*   **Services:** KasmVNC (Desktop GUI), Jupyter Lab (this interface), and Caddy (Web Portal) are running in the background.

---## Section 1: Setup Conda Environment (`viso_env`)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import subprocess
import time
import os

# --- Helper Function (assuming it's defined elsewhere or copy it here) ---
# Make sure the run_command function from the previous notebook version is included here
def run_command(command, output_widget, status_widget, success_message, failure_message):
    status_widget.value = "<i>Status: Running...</i>"
    output_widget.clear_output()
    with output_widget:
        print(f"Running command: {' '.join(command)}\n---")
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True)
        
        stdout_lines = []
        while True:
            line = process.stdout.readline()
            if not line:
                break
            print(line.strip()) # Print line by line to the output widget
            stdout_lines.append(line.strip())
        
        process.wait()
        print("---")
        if process.returncode == 0:
            print(f"✅ Success: {success_message}")
            status_widget.value = f"<i>Status: ✅ {success_message}</i>"
            return True, "\n".join(stdout_lines)
        else:
            print(f"❌ Error: {failure_message} (Exit Code: {process.returncode})")
            status_widget.value = f"<i>Status: ❌ {failure_message}</i>"
            return False, "\n".join(stdout_lines)

# --- Widgets --- 
env_output = widgets.Output()
env_status = widgets.HTML(value="<i>Status: Idle</i>")
cuda_output = widgets.Output()
cuda_status = widgets.HTML(value="<i>Status: Idle</i>")
verify_output = widgets.Output()
verify_status = widgets.HTML(value="<i>Status: Idle</i>")

create_env_button = widgets.Button(description="1. Create/Check 'viso_env' (Python 3.10)", button_style='info', layout=widgets.Layout(width='95%'))
install_cuda_button = widgets.Button(description="2. Install Conda CUDA 12.4.1 & cuDNN", button_style='info', layout=widgets.Layout(width='95%'), disabled=True)
verify_env_button = widgets.Button(description="3. Verify Environment", button_style='info', layout=widgets.Layout(width='95%'), disabled=True)

# --- Functions --- 
def check_env_exists(env_name='viso_env'):
    result = subprocess.run(['conda', 'info', '--envs'], capture_output=True, text=True)
    return f"\n{env_name} " in result.stdout

# --- Button Actions --- 
@create_env_button.on_click
def on_create_env_click(b):
    create_env_button.disabled = True
    install_cuda_button.disabled = True
    verify_env_button.disabled = True
    if check_env_exists():
        env_status.value = "<i>Status: 'viso_env' already exists. Checking Python version...</i>"
        with env_output:
            env_output.clear_output()
            print("'viso_env' already exists.")
            # Optional: Add check for Python version here if needed
            print("✅ Proceeding to next step.")
        env_status.value = "<i>Status: ✅ 'viso_env' exists.</i>"
        install_cuda_button.disabled = False
    else:
        command = ['conda', 'create', '-n', 'viso_env', 'python=3.10', '-y']
        success, _ = run_command(command, env_output, env_status, "'viso_env' created successfully.", "Failed to create 'viso_env'.")
        if success:
            install_cuda_button.disabled = False 
    create_env_button.disabled = False # Re-enable after completion

@install_cuda_button.on_click
def on_install_cuda_click(b):
    install_cuda_button.disabled = True
    verify_env_button.disabled = True

    status_widget = cuda_status
    output_widget = cuda_output
    status_widget.value = "<i>Status: Installing CUDA Runtime...</i>"
    output_widget.clear_output()

    # Install CUDA Runtime
    cmd_cuda = ['conda', 'install', '-n', 'viso_env', '-c', 'nvidia/label/cuda-12.4.1', 'cuda-runtime', '-y']
    cuda_success, _ = run_command(cmd_cuda, output_widget, status_widget, "CUDA Runtime installed.", "Failed to install CUDA Runtime.")

    if cuda_success:
        status_widget.value = "<i>Status: Installing cuDNN...</i>"
        # Install cuDNN
        cmd_cudnn = ['conda', 'install', '-n', 'viso_env', '-c', 'conda-forge', 'cudnn', '-y']
        cudnn_success, _ = run_command(cmd_cudnn, output_widget, status_widget, "cuDNN installed.", "Failed to install cuDNN.")
        if cudnn_success:
             status_widget.value = "<i>Status: ✅ CUDA Runtime & cuDNN installed.</i>"
             verify_env_button.disabled = False
        else:
             status_widget.value = "<i>Status: ❌ Failed to install cuDNN.</i>"
    else:
        status_widget.value = "<i>Status: ❌ Failed to install CUDA Runtime.</i>"
        
    install_cuda_button.disabled = False # Re-enable after completion

@verify_env_button.on_click
def on_verify_env_click(b):
    verify_env_button.disabled = True
    command = ['conda', 'list', '-n', 'viso_env']
    success, output = run_command(command, verify_output, verify_status, "Environment details listed.", "Failed to list environment details.")
    with verify_output:
        if success:
            # Simple string checks - might need refinement if versions are very specific
            py_found = any('python' in line and '3.10' in line for line in output.splitlines())
            cuda_found = any('cuda-runtime' in line and ('12.4' in line or '12.4.1' in line) for line in output.splitlines())
            cudnn_found = any('cudnn' in line for line in output.splitlines())
            
            if py_found and cuda_found and cudnn_found:
                 print("\n✅ Verification successful: Python 3.10, CUDA Runtime 12.4.x, and cuDNN found.")
                 verify_status.value = "<i>Status: ✅ Verification successful.</i>"
            else:
                 missing = []
                 if not py_found: missing.append('Python 3.10')
                 if not cuda_found: missing.append('CUDA Runtime 12.4.x')
                 if not cudnn_found: missing.append('cuDNN')
                 print(f"\n⚠️ Verification warning: Could not confirm presence of: {', '.join(missing)}. Check output above.")
                 verify_status.value = f"<i>Status: ⚠️ Verification warning. Missing: {', '.join(missing)}.</i>"
        else:
            print("\n❌ Verification failed.")
    verify_env_button.disabled = False # Re-enable after completion

# --- Display Widgets --- 
display(HTML("<b>Step 1.1: Create Conda Environment 'viso_env'</b>"))
display(create_env_button)
display(env_status)
display(env_output)

display(HTML("<b>Step 1.2: Install CUDA Runtime & cuDNN into 'viso_env'</b>"))
display(install_cuda_button)
display(cuda_status)
display(cuda_output)

display(HTML("<b>Step 1.3: Verify Environment Contents</b>"))
display(verify_env_button)
display(verify_status)
display(verify_output)

---
## Section 2: Install Python Dependencies (Pip)

In [ ]:
# --- Widgets and Functions from Previous Cell (Section 1) --- 
# Ensure run_command is defined or imported
# --- Configuration ---
requirements_file = "/workspace/requirements.txt" # Standard name
pytorch_index = "https://download.pytorch.org/whl/cu124"
nvidia_index = "https://pypi.nvidia.com"

# --- Widgets ---
pip_output = widgets.Output()
pip_status = widgets.HTML(value="<i>Status: Idle</i>")
install_pip_button = widgets.Button(description=f"Install packages from {os.path.basename(requirements_file)}", button_style='info', layout=widgets.Layout(width='95%'))
file_check_status = widgets.HTML()

# --- Functions ---
def check_requirements_file():
    if os.path.exists(requirements_file):
        file_check_status.value = f"✅ Found '{os.path.basename(requirements_file)}' in /workspace."
        install_pip_button.disabled = False
    else:
        file_check_status.value = f"❌ **Action Required:** Please upload your `requirements.txt` file to the `/workspace` directory using the Jupyter Lab file browser."
        install_pip_button.disabled = True

# --- Button Actions ---
@install_pip_button.on_click
def on_install_pip_click(b):
    install_pip_button.disabled = True
    check_requirements_file() # Re-check just before running
    if not install_pip_button.disabled:
        command = [
            '/opt/conda/envs/viso_env/bin/python', '-m', 'pip', 'install',
            '-r', requirements_file,
            '--extra-index-url', pytorch_index,
            '--extra-index-url', nvidia_index,
            '--no-cache-dir' # Avoid caching issues
        ]
        success, _ = run_command(command, pip_output, pip_status, "Pip packages installed successfully.", "Failed to install pip packages.")
        if success:
            # Optionally add a verification step here (e.g., pip list)
            pass
    install_pip_button.disabled = False # Re-enable

# --- Initial Check and Display ---
display(HTML(f"<b>Step 2.1: Check for requirements file</b><br>This step installs Python packages using pip. Ensure you have a `requirements.txt` file in your `/workspace` directory. This file should contain the packages needed by VisoMaster, like `torch`, `onnxruntime-gpu`, etc."))
display(file_check_status)
check_requirements_file() # Initial check

display(HTML("<b>Step 2.2: Install Pip Packages into 'viso_env'</b>"))
display(install_pip_button)
display(pip_status)
display(pip_output)

---
## Section 3: VisoMaster Code, Assets, and Models

In [ ]:
# --- Widgets and Functions from Previous Cell (Section 1) --- 
# Ensure run_command is defined or imported
# --- Configuration ---
visomaster_dir = "/workspace/VisoMaster" # Assumes user clones/uploads here
dependencies_dir = "/workspace/dependencies" # Assumes user uploads assets here
models_dir = "/workspace/models" # Output for download script
download_script = os.path.join(visomaster_dir, "download_models.py")

# --- Widgets ---
code_check_status = widgets.HTML()
deps_check_status = widgets.HTML()
model_output = widgets.Output()
model_status = widgets.HTML(value="<i>Status: Idle</i>")
download_models_button = widgets.Button(description="Download VisoMaster Models", button_style='info', layout=widgets.Layout(width='95%'))

# --- Functions ---
def check_visomaster_files():
    code_exists = os.path.isdir(visomaster_dir) and os.path.exists(download_script)
    # Check if deps_dir exists and is not empty, handle case where it doesn't exist gracefully
    deps_exist = False
    if os.path.isdir(dependencies_dir):
        try:
            if len(os.listdir(dependencies_dir)) > 0:
                deps_exist = True
        except OSError: # Handle potential permission errors
            deps_exist = False 
            
    if code_exists:
        code_check_status.value = f"✅ Found VisoMaster code and download script in '{visomaster_dir}'."
    else:
        code_check_status.value = f"❌ **Action Required:** Please clone or upload the VisoMaster application code into `/workspace/VisoMaster`. Ensure `download_models.py` is present."

    if deps_exist:
        deps_check_status.value = f"✅ Found non-empty 'dependencies' folder in `/workspace`."
    else:
        deps_check_status.value = f"⚠️ **Action Required:** Please create a `dependencies` folder in `/workspace` and upload the required VisoMaster assets (e.g., from the releases page, *excluding* source code zip) into it."

    download_models_button.disabled = not (code_exists) # Only enable if code/script exists

# --- Button Actions ---
@download_models_button.on_click
def on_download_models_click(b):
    download_models_button.disabled = True
    check_visomaster_files() # Re-check
    if not download_models_button.disabled:
        # Ensure model output directory exists
        os.makedirs(models_dir, exist_ok=True)
        
        command = [
            '/opt/conda/envs/viso_env/bin/python',
            download_script,
            '--output_dir', models_dir
        ]
        success, _ = run_command(command, model_output, model_status, "Model download script finished.", "Model download script failed.")
        # Add check if models_dir is non-empty after success if needed
        if success:
             try:
                 if len(os.listdir(models_dir)) > 0:
                      model_status.value = f"<i>Status: ✅ Model download script finished. Found files in {models_dir}.</i>"
                 else:
                      model_status.value = f"<i>Status: ⚠️ Model download script finished, but {models_dir} appears empty. Check script output.</i>"
             except OSError:
                  model_status.value = f"<i>Status: ⚠️ Model download script finished, but could not check {models_dir}. Check script output.</i>"
    download_models_button.disabled = False # Re-enable

# --- Initial Check and Display ---
display(HTML("<b>Step 3.1: Check for VisoMaster Code and Assets</b><br>VisoMaster requires its source code and specific asset files."))
display(HTML("<ul><li>Clone or upload the VisoMaster repository into <code>/workspace/VisoMaster</code>.</li><li>Create <code>/workspace/dependencies</code> and upload required assets from the VisoMaster assets release page inside it.</li></ul>"))
display(code_check_status)
display(deps_check_status)
check_visomaster_files() # Initial check

display(HTML(f"<b>Step 3.2: Download Models</b><br>This will run the `download_models.py` script found in your VisoMaster code directory to download model files into <code>{models_dir}</code>."))
display(download_models_button)
display(model_status)
display(model_output)

---
## Section 4: Launch VisoMaster in KasmVNC

**VisoMaster is a GUI application and needs to run within the KasmVNC desktop environment.**

**Instructions:**
1.  **Access KasmVNC:** Open the Caddy Portal (using the 'Open' button on the Vast.ai instances page) and click the link for 'VNC'.
2.  **Open Terminal:** Inside the KasmVNC desktop, find and open a terminal application (e.g., look for 'Terminal', 'Xterm', or similar in the application menu).
3.  **Run Command:** In the KasmVNC terminal, type or paste the following command and press Enter:

```bash
# Activate the conda environment
conda activate viso_env

# Navigate to the VisoMaster code directory
cd /workspace/VisoMaster

# Run the application (ensure paths to models/dependencies are correct)
# Add any other required command-line arguments for VisoMaster
python app.py --models_dir /workspace/models --dependencies_dir /workspace/dependencies
```

4.  The VisoMaster application window should appear within the KasmVNC desktop.

*(Note: Launching a GUI application directly from this Jupyter notebook is complex. Running the command inside the KasmVNC terminal is the recommended method.)*

---
## Section 5: Troubleshooting & Monitoring

In [ ]:
# --- Widgets and Functions from Previous Cell (Section 1) --- 
# Ensure run_command is defined or imported
# --- Widgets --- 
logs_output = widgets.Output()
logs_status = widgets.HTML(value="<i>Status: Idle</i>")
status_output = widgets.Output()
status_status = widgets.HTML(value="<i>Status: Idle</i>")
gpu_output = widgets.Output()
gpu_status = widgets.HTML(value="<i>Status: Idle</i>")

show_logs_button = widgets.Button(description="Show Service Logs (tail)", button_style='warning', layout=widgets.Layout(width='95%'))
check_status_button = widgets.Button(description="Check Service Status (supervisorctl)", button_style='warning', layout=widgets.Layout(width='95%'))
check_gpu_button = widgets.Button(description="Check GPU Availability (nvidia-smi & PyTorch)", button_style='warning', layout=widgets.Layout(width='95%'))

# --- Button Actions --- 
@show_logs_button.on_click
def on_show_logs_click(b):
    # Show last 50 lines of common log files
    command = ['tail', '-n', '50', '/var/log/supervisor/supervisord.log', 
               '/var/log/supervisor/jupyterlab_stdout.log', '/var/log/supervisor/jupyterlab_stderr.log',
               '/var/log/supervisor/kasmvnc_stdout.log', '/var/log/supervisor/kasmvnc_stderr.log',
               '/var/log/supervisor/caddy_stdout.log', '/var/log/supervisor/caddy_stderr.log']
    run_command(command, logs_output, logs_status, "Displayed logs.", "Failed to display logs.")
    with logs_output:
         print("\n--- Note: For full logs, check the '/logs/' link in the Caddy Portal or browse '/var/log/supervisor/' using the Jupyter file browser. ---")

@check_status_button.on_click
def on_check_status_click(b):
    command = ['supervisorctl', 'status']
    success, output = run_command(command, status_output, status_status, "Displayed service status.", "Failed to get service status.")
    with status_output:
        if success:
            print("\n--- Expected Status: RUNNING for caddy, jupyterlab, kasmvnc ---")
            # Removed clone_notebook check
            print("\nTo restart a service: supervisorctl restart <program_name> (e.g., supervisorctl restart kasmvnc)")

@check_gpu_button.on_click
def on_check_gpu_click(b):
    gpu_status.value = "<i>Status: Running nvidia-smi...</i>"
    gpu_output.clear_output()
    # Run nvidia-smi
    cmd_smi = ['nvidia-smi']
    smi_success, _ = run_command(cmd_smi, gpu_output, gpu_status, "nvidia-smi finished.", "Failed to run nvidia-smi.")
    
    if smi_success:
        # Check if viso_env exists before trying to use it
        if check_env_exists('viso_env'): 
            gpu_status.value = "<i>Status: Checking PyTorch CUDA availability...</i>"
            # Check PyTorch in viso_env
            cmd_torch = [
                '/opt/conda/envs/viso_env/bin/python', '-c',
                'import torch; print(f"PyTorch CUDA Available in viso_env: {torch.cuda.is_available()}"); print(f"Device Count: {torch.cuda.device_count()}")'
            ]
            torch_success, _ = run_command(cmd_torch, gpu_output, gpu_status, "PyTorch check finished.", "Failed to run PyTorch check.")
            if torch_success:
                 gpu_status.value = "<i>Status: ✅ GPU checks complete.</i>"
            else:
                 gpu_status.value = "<i>Status: ⚠️ nvidia-smi succeeded, but PyTorch check failed. Ensure PyTorch GPU version is installed correctly in viso_env.</i>"
        else:
            gpu_status.value = "<i>Status: ⚠️ nvidia-smi succeeded, but 'viso_env' does not exist yet. Run Section 1 first.</i>"
            with gpu_output:
                 print("\n--- Cannot check PyTorch CUDA status because 'viso_env' has not been created. Please run Section 1 first. ---")
    else:
        gpu_status.value = "<i>Status: ❌ nvidia-smi failed. GPU drivers might not be available or working correctly on the host.</i>"

# --- Display Widgets --- 
display(HTML("<b>Check Service Logs</b>"))
display(show_logs_button)
display(logs_status)
display(logs_output)

display(HTML("<b>Check Service Status</b>"))
display(check_status_button)
display(status_status)
display(status_output)

display(HTML("<b>Check GPU Status</b>"))
display(check_gpu_button)
display(gpu_status)
display(gpu_output)